In [1]:
# Develop simulated football season.
# Connect to a Java Neural Network using Lasagne.
# http://lasagne.readthedocs.org/
#    More in-depth examples and reproductions of paper results are maintained in
#    a separate repository: https://github.com/Lasagne/Recipes
# 2022-01-17

In [1]:
# Lasagne

from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

In [3]:
# Football team class
class Team:
    """Class holding team information"""

    nextTeamId = 0
    minYear = 1990
    
    teamObjectById = {}
    teamObjectByName = {}
    
    # inactiveName = "INACTIVE" # If this string is returned for conference, division or team name then that team is currently inactive.
    
    @staticmethod
    def IsTeamActiveById(teamId, year):
        if (teamId >= Team.nextTeamId):
            return False
        if (Team.teamObjectById[teamId].schedule.get(year) == None):
            return False
        return True
    
    @staticmethod
    def GetListOfTeamIds(year):
        listOfTeamIds = []
        for teamId in range(Team.nextTeamId):
            if (Team.IsTeamActiveById(teamId, year)):
                listOfTeamIds.append(teamId)
        return listOfTeamIds

    # Note: if a team does not play for a given year, set conferenceName and divisionName to "unassigned".
    
    def __init__(self, teamName, startYear=minYear, conferenceName=None, divisionName=None):
        self.teamName = teamName
        self.teamId = Team.nextTeamId
        Team.teamObjectById[self.teamId] = self
        Team.teamObjectByName[self.teamName] = self
        Team.nextTeamId += 1
        self.conferenceName = {}
        self.divisionName = {}
        self.defensePower = {}
        self.offensePower = {}
        self.defensePowerActual = {} # For simulated seasons, the "actual" power for the given year
        self.offensePowerActual = {} # For simulated seasons, the "actual" power for the given year
        self.fitPowerPerYearPerRound = {}
        self.schedule = {}
        self.homeField = {} # 1-home, 0.5-neutral, 0-away
        self.startYear = startYear
        if (conferenceName != None):
            self.conferenceName[startYear] = conferenceName
        if (divisionName != None):
            self.divisionName[startYear] = divisionName

    def SetConferenceAndDivision(self, year, conference, division):
        self.conferenceName[year] = conference
        self.divisionName[year] = division
            
    def GetConferenceName(self, year):
        yearCheck = year
        while(self.conferenceName.get(yearCheck) == None and yearCheck > Team.minYear):
            yearCheck = yearCheck - 1
        return self.conferenceName.get(yearCheck)
 
    def GetDivisionName(self, year):
        yearCheck = year
        while(self.divisionName.get(yearCheck) == None and yearCheck > Team.minYear):
           yearCheck = yearCheck - 1
        return self.divisionName.get(yearCheck)

    def SetPower(self, year, defensePower, offensePower):
        # Current power for a given season using best estimate for scores through latest week.
        self.defensePower[year] = defensePower
        self.offensePower[year] = offensePower

    def GetOffensePower(self, year):
        return self.offensePower[year]
   
    def GetDefensePower(self, year):
        return self.defensePower[year]
    
    def GetPower(self, year):
        power = {}
        power["defense"] = self.defensePower[year]
        power["offense"] = self.offensePower[year]
        return power
        
    def SetPowerActual(self, year, defensePower, offensePower):
        # Actual power is the real truth power for a simulated season.  It is used when testing which weights to use to get current power for a given week.
        self.defensePowerActual[year] = defensePower
        self.offensePowerActual[year] = offensePower

    def GetPowerActual(self, year):
        # Actual power is the real truth power for a simulated season.  It is used when testing which weights to use to get current power for a given week.
        actualPower = {}
        actualPower["offense"] = self.offensePowerActual[year]
        actualPower["defense"] = self.defensePowerActual[year]
        return actualPower
    
    def SetFitPower(self, year, roundName, offense, defense):
        # This is the best estimate for team power after averaging fit power for all rounds up through the given round.
        if (self.fitPowerPerYearPerRound.get(year) == None):
            self.fitPowerPerYearPerRound[year] = {}
        if (self.fitPowerPerYearPerRound[year].get(roundName) == None):
            self.fitPowerPerYearPerRound[year][roundName] = {}
        self.fitPowerPerYearPerRound[year][roundName]["offense"] = offense
        self.fitPowerPerYearPerRound[year][roundName]["defense"] = defense   
        
    def GetFitPower(self, year, roundName):
        # This is the best estimate for team power after averaging fit power for all rounds up through the given round.
        if (self.fitPowerPerYearPerRound.get(year) == None 
            or self.fitPowerPerYearPerRound[year].get(roundName) == None):
            return None
            #print("::: fit power for "+str(year)+" "+roundName+" not found.  Returning general power values.")
            #power = {}
            #power["offense"] = self.offensePower[year]
            #power["defense"] = self.defensePower[year]
            #return power
        else:
            return self.fitPowerPerYearPerRound[year][roundName]     
        
    def SetOpponent(self, year, roundName, opponentName, homeField):
        if (self.schedule.get(year) == None):
            self.schedule[year] = {}
        self.schedule[year][roundName] = opponentName
        if (self.homeField.get(year) == None):
            self.homeField[year] = {}
        self.homeField[year][roundName] = homeField

    def GetOpponent(self, year, roundName):
        if (self.schedule.get(year) == None):
            return None
        return self.schedule[year].get(roundName)

    def GetHomeField(self, year, roundName):
        if (self.homeField.get(year) == None):
            return None
        return self.schedule[year].get(roundName)
        
#    @staticmethod
#    def AddTeamInfo(teamObject, teamId, teamName):
#        sample static method
        
#print(Team.nextTeamId)
#sampleTeam = Team('BYU')
#sampleTeam.SetConferenceAndDivision(2021, "Independent", "IA")
#print(Team.nextTeamId)
#print(sampleTeam.conferenceName[2021])
#print("again")
#print(sampleTeam.GetConferenceName(2022))

In [16]:
import math
from scipy import special

# from checkSpreadVsProb which has a collection of 65-75% BFM predictions which has an average spread of 13.4.
prob = 0.5*(1+special.erf(13.4/25/math.sqrt(2)))
print("prob "+str(prob))

# prob 0.7040207247756906
# => rmsFactor = 25

prob 0.7040207247756906


In [4]:
import math
from scipy import special

class GameSimulator:
    """Class for generating results/predictions of games"""

    def __init__(self, homeAdvantage, spdCoefOff, spdCoefDef, totCoef0, totCoefOff, totCoefDef, rmsCoef0, rmsCoefOff, rmsCoefDef, maxConfGameWeights):
        self.homeFieldAdvantage = homeAdvantage
        self.spdCoefOff = spdCoefOff
        self.spdCoefDef = spdCoefDef
        self.totCoef0 = totCoef0
        self.totCoefOff = totCoefOff
        self.totCoefDef = totCoefDef
        self.rmsCoef0 = rmsCoef0
        self.rmsCoefOff = rmsCoefOff
        self.rmsCoefDef = rmsCoefDef
        self.weights = {}
        self.deltaWeights = {}
        self.weights["conference"] = []
        self.weights["nonconference"] = []
        self.weights["nondivision"] = []
        self.deltaWeights["conference"] = []
        self.deltaWeights["nonconference"] = []
        self.deltaWeights["nondivision"] = []
        while (len(self.weights["conference"]) < maxConferenceGameWeights):
            self.weights["conference"].append(1.)
            self.weights["nonconference"].append(1.)
            self.weights["nondivision"].append(1.)
            self.deltaWeights["conference"].append(0.)
            self.deltaWeights["nonconference"].append(0.)
            self.deltaWeights["nondivision"].append(0.)
    
    def RandomizeWeights(self):
        for iCount in range(len(self.weights["conference"])): 
            self.weights["conference"][iCount] = random.random()
            self.weights["nondivision"][iCount] = random.random()
            self.weights["nonconference"][iCount] = random.random()

    def SetWeights(self, value=1):
        for iCount in range(len(self.weights["conference"])): 
            self.weights["conference"][iCount] = value
            self.weights["nondivision"][iCount] = value
            self.weights["nonconference"][iCount] = value
            
    def ResetDeltaWeights(self):
        for iCount in range(len(self.weights["conference"])): 
            self.deltaWeights["conference"][iCount] = 0
            self.deltaWeights["nondivision"][iCount] = 0
            self.deltaWeights["nonconference"][iCount] = 0
            
    def ApplyDeltaWeights(self):
        for iCount in range(len(self.weights["conference"])): 
            self.weights["conference"][iCount] += self.deltaWeights["conference"][iCount]
            self.weights["nondivision"][iCount] += self.deltaWeights["nondivision"][iCount]
            self.weights["nonconference"][iCount] += self.deltaWeights["nonconference"][iCount]
    
    def RandomizeLinearParameters(self):
        self.homeFieldAdvantage = 5*random.random()
        self.spdCoefOff = 5+30*random.random()
        self.spdCoefDef = 5+30*random.random() 
        self.self.totCoef0 = -50+100*random.random()
        self.totCoefOff = 4+20*random.random()
        self.totCoefDef = -4-20*random.random()
        #rmsCoef0 = 16.173 
        #rmsCoefOff = 1.324 
        #rmsCoefDef = -1.684
            
    def GetScoreAndProbability(self, o1, d1, o2, d2, homeField):
        # Return the average score and probability expected for the given power and home field.
        results = {}
        s1 = 0.5*(totCoefOff*(o1+o2) + totCoefDef*(d1+d2) + totCoef0 + spdCoefOff*(o1-o2) + spdCoefDef*(d1-d2))
        s2 = 0.5*(totCoefOff*(o2+o1) + totCoefDef*(d2+d1) + totCoef0 + spdCoefOff*(o2-o1) + spdCoefDef*(d2-d1))
        if (homeField == 1.):
            s1 += self.homeFieldAdvantage/2.
            s2 -= self.homeFieldAdvantage/2.
        if (homeField == 0.):
            s1 -= self.homeFieldAdvantage/2.
            s2 += self.homeFieldAdvantage/2.
        if (s1 < 0):
            s1 = 0.
        if (s2 < 0):
            s2 = 0.
        rms = rmsCoef0 + (o1+o2)*rmsCoefOff + (d1+d2)*rmsCoefDef
        spread = s1 - s2
        prob = 0.5*(1+special.erf(spread/rms/math.sqrt(2)))
        results["score1"] = s1
        results["score2"] = s2
        results["probability"] = prob
        results["scoreRms"] = rms
        return results
    
    def GetScores(self, o1, d1, o2, d2, homeField):
        # Return the average score and probability expected for the given power and home field.
        results = {}
        s1 = 0.5*(totCoefOff*(o1+o2) + totCoefDef*(d1+d2) + totCoef0 + spdCoefOff*(o1-o2) + spdCoefDef*(d1-d2))
        s2 = 0.5*(totCoefOff*(o2+o1) + totCoefDef*(d2+d1) + totCoef0 + spdCoefOff*(o2-o1) + spdCoefDef*(d2-d1))
        if (homeField == 1.):
            s1 += self.homeFieldAdvantage/2.
            s2 -= self.homeFieldAdvantage/2.
        if (homeField == 0.):
            s1 -= self.homeFieldAdvantage/2.
            s2 += self.homeFieldAdvantage/2.
        if (s1 < 0):
            s1 = 0.
        if (s2 < 0):
            s2 = 0.
        results["score1"] = s1
        results["score2"] = s2
        return results
    
    def GetRandomizedScore(self, o1, d1, o2, d2, homeField):
        # Return the average score and probability expected for the given power and home field.
        averageResults = self.GetScoreAndProbability(o1, d1, o2, d2, homeField)
        # rms for the spread
        # scoreRms = rmsCoef0 + (o1+o2)*rmsCoefOff + (d1+d2)*rmsCoefDef  
        # RMS for the score of one team - believe this should be spreadRms/sqrt(2)
        #fudgeFactor = 1.40
        #scoreRms = spreadRms/math.sqrt(2.)*fudgeFactor
        # From "Test statistic for randomizing the scores." below, rms for one team works out to be spreadRms.
        scoreRms = averageResults["scoreRms"]
        s1 = scoreRms*special.erfinv(2*random.random()-1.) + averageResults["score1"]
        s2 = scoreRms*special.erfinv(2*random.random()-1.) + averageResults["score2"]
        if (homeField == 1.):
            s1 += self.homeFieldAdvantage/2.
            s2 -= self.homeFieldAdvantage/2.
        if (homeField == 0.):
            s1 -= self.homeFieldAdvantage/2.
            s2 += self.homeFieldAdvantage/2.
        if (s1 < 0.):
            s1 = 0.
        if (s2 < 0.):
            s2 = 0.
        s1 = int(s1+0.5)
        s2 = int(s2+0.5)
        randomizedScore = {}
        randomizedScore["score1"] = s1
        randomizedScore["score2"] = s2
        randomizedScore["averageScore1"] = averageResults["score1"]
        randomizedScore["averageScore2"] = averageResults["score2"]
        randomizedScore["probability"] = averageResults["probability"]
        return randomizedScore

    def AdjustPowerToFitActualScore(self, o1, d1, o2, d2, homeField, actualScore1, actualScore2):
        # Linear model equations:
        # 
        #    alpha = spd_coef_def/spd_coef_off 
        #    beta = tot_coef_def/tot_coef_off
        # 
        #    spread =  s1 - s2 = spd_coef_off*(o1-o2) + spd_coef_def*(d1-d2)
        #    spread = spd_coef_off*(o1-o2 + alpha*(d1-d2))
        # 
        #    total = s1 + s2 = tot_coef_off*(o1+o2) + tot_coef_off*(d1+d2)
        #    total = s1 + s2 = tot_coef_off*(o1+o2 + beta*(d1+d2))
        # 
        # Equations for adjusting actual vs expected power using actual scores
        # 
        #    Actual spread and total points:
        # 
        #       spread_actual = spd_coef_off*[o1_adjusted-o2_adjusted + alpha*(d1_adjusted-d2_adjusted)]
        #       total_actual = tot_coef_off*[o1_adjusted+o2_adjusted + beta*(d1_adjusted+d2_adjusted)]
        # 
        #    Constraint - total power remains the same:
        # 
        #       o1_orig+alpha*d1_orig + o2_orig+alpha*d2_orig = o1_adjusted+alpha*d1_adjusted + o2_adjusted+alpha*d2_adjusted 
        # 
        #    Constraint - minimize the changes in power:
        # 
        #       changes in power = (o1_orig-o1_adjusted)^2 + alpha^2*(d1_orig-d1_adjusted)^2 + (o2_orig-o2_adjusted)^2 + alpha^2*(d2_orig-d2_adjusted)^2 
        #       d change / d d1_adjusted = 2*[(o1_orig-o1_adjusted)*d o1_adjusted/d d1_adjusted + alpha^2*(d1_orig-d1_adjusted)*d d1_adjusted/d d1_adjusted + (o2_orig-o2_adjusted)*d o2_adjusted/d d1_adjusted + alpha^2*(d2_orig-d2_adjusted)*d d2_adjusted/d d1_adjusted] = 0
        #       (o1_orig-o1_adjusted)*d o1_adjusted/d d1_adjusted + alpha^2*(d1_orig-d1_adjusted)*d d1_adjusted/d d1_adjusted + (o2_orig-o2_adjusted)*d o2_adjusted/d d1_adjusted + alpha^2*(d2_orig-d2_adjusted)*d d2_adjusted/d d1_adjusted = 0
        #
        # Solved equations with maxima: https://home.csulb.edu/~woollett/mbe4solve.pdf
        # See bcm/notes_linear.txt

        adjustedPower = {}
        
        o1Orig = o1
        d1Orig = d1
        o2Orig = o2
        d2Orig = d2
        
        # Fit is done on neutral field.  Remove home field advantage before adjusting.
        if (homeField == 0.):
            actualScore1 += self.homeFieldAdvantage/2.
            actualScore2 -= self.homeFieldAdvantage/2.
        if (homeField == 1.):
            actualScore1 -= self.homeFieldAdvantage/2.
            actualScore2 += self.homeFieldAdvantage/2.
        
        expectedScore = self.GetScoreAndProbability(o1, d1, o2, d2, 0.5)
        
        spreadDelta = expectedScore["score1"] - expectedScore["score2"] - actualScore1 + actualScore2
        totalDelta = expectedScore["score1"] + expectedScore["score2"] - actualScore1 - actualScore2
               
        alpha = spdCoefDef/spdCoefOff 
        beta = totCoefDef/totCoefOff
        
        # o1Actual = o1Orig + o1Delta
        # d1Actual = d1Orig + d1Delta
        # o2Actual = o2Orig + o2Delta
        # d2Actual = d2Orig + d2Delta

        alpha = spdCoefDef/spdCoefOff 
        beta = totCoefDef/totCoefOff

        # Spread delta = expected spread - actual spread
        # spreadDelta = spdCoefOff*(o1Delta-o2Delta + alpha*(d1Delta-d2Delta))
        
        # Total delta = expected total - actual total
        # totalDelta = totCoefOff*(o1Delta+o2Delta + beta*(d1Delta+d2Delta))
        # 
        # Constraint - total power remains the same:
        # o1Delta + alpha*d1Delta + o2Delta + alpha*d2Delta = 0
        # 
        # Constraint - minimize the change in power:
        # Change in power = o1Delta^2 + (alpha*d1Delta)^2 + o2Delta^2 + (alpha*d2Delta)^2
        # o1Delta - o2Delta + alpha*(d2Delta - d1Delta) = 0

        o1Delta = -(2*alpha*spdCoefOff*totalDelta+spreadDelta*totCoefOff*(alpha-beta))/(spdCoefOff*4*totCoefOff*(alpha-beta))
        d1Delta = -(spreadDelta*totCoefOff*(alpha-beta)-2*alpha*spdCoefOff*totalDelta)/(spdCoefOff*4*alpha*totCoefOff*(alpha-beta))
        o2Delta = (spreadDelta*totCoefOff*(alpha-beta)-2*alpha*spdCoefOff*totalDelta)/(spdCoefOff*4*totCoefOff*(alpha-beta))
        d2Delta = (2*alpha*spdCoefOff*totalDelta+spreadDelta*totCoefOff*(alpha-beta))/(spdCoefOff*4*alpha*totCoefOff*(alpha-beta))

        #print("::: deltas "+str(o1Delta)+" "+str(d1Delta)+" "+str(o2Delta)+" "+str(d2Delta))
        o1Adjusted = o1Orig + o1Delta
        d1Adjusted = d1Orig + d1Delta
        o2Adjusted = o2Orig + o2Delta
        d2Adjusted = d2Orig + d2Delta
        
        adjustedPower["offense1"] = o1Adjusted
        adjustedPower["defense1"] = d1Adjusted
        adjustedPower["offense2"] = o2Adjusted
        adjustedPower["defense2"] = d2Adjusted
        
        if (False):
            # debug info:
            origPower = o1+alpha*d1 + o2+alpha*d2
            newPower = o1Adjusted+alpha*d1Adjusted + o2Adjusted+alpha*d2Adjusted

            verifyAdjusted = self.GetScoreAndProbability(o1Adjusted, d1Adjusted, o2Adjusted, d2Adjusted, 0.5)
            print("::: o&d "+str(o1)+" "+str(d1)+" "+str(o2)+" "+str(d2) 
                  +" neutral expected "+str(expectedScore["score1"])+"-"+str(expectedScore["score2"]))
            print("    neutral actual "+str(actualScore1)+" "+str(actualScore2)+" adj o&d "
                  +str(o1Adjusted)+" "+str(d1Adjusted)+" "+str(o2Adjusted)+" "+str(d2Adjusted)
                  +" verify "+str(verifyAdjusted["score1"])+" "+str(verifyAdjusted["score2"]))
            print("   origPower "+str(origPower)+" adjustedPower "+str(newPower))
            
        return adjustedPower
        

homeAdvantage = 3.
spdCoefOff = 18.4
spdCoefDef = 17.6 
totCoef0 = -17 
totCoefOff = 14.8 
totCoefDef = -9.6 
rmsCoef0 = 16.173 
rmsCoefOff = 1.324 
rmsCoefDef = -1.684
maxConferenceGameWeights = 25

gameSimulator = GameSimulator(homeAdvantage, spdCoefOff, spdCoefDef, totCoef0, totCoefOff, totCoefDef, rmsCoef0, rmsCoefOff, rmsCoefDef, maxConferenceGameWeights)

In [5]:
# Test fit

o1 = 10.
d1 = 8.
o2 = 9.
d2 = 7.
expectedScoreNeutral = gameSimulator.GetScoreAndProbability(o1, d1, o2, d2, 0.5)
print("::: expectedScoreNeutral "+str(expectedScoreNeutral))
expectedScoreHome = gameSimulator.GetScoreAndProbability(o1, d1, o2, d2, 1.)
print("::: expectedScoreNeutral "+str(expectedScoreNeutral))
print("")
#expectedScore = gameSimulator.GetScoreAndProbability(o1, d1, o2, d2, 1.)
#print("::: expectedScore "+str(expectedScore))
gameSimulator.AdjustPowerToFitActualScore(o1, d1, o2, d2, 0.5, 50., 45.)
print("")
#gameSimulator.AdjustPowerToFitActualScore(o1, d1, o2, d2, 1., 51.5, 43.5)
gameSimulator.AdjustPowerToFitActualScore(o1, d1, o2, d2, 0.5, 79.6, 42.1)
print("")
gameSimulator.AdjustPowerToFitActualScore(o1, d1, o2, d2, 1., 79.6+1.5, 42.1-1.5)

::: expectedScoreNeutral {'score1': 78.1, 'score2': 42.099999999999994, 'probability': 0.9874655234980421, 'scoreRms': 16.069000000000003}
::: expectedScoreNeutral {'score1': 78.1, 'score2': 42.099999999999994, 'probability': 0.9874655234980421, 'scoreRms': 16.069000000000003}





{'offense1': 10.05057809217646,
 'defense1': 7.989736539997338,
 'offense2': 9.009817222611241,
 'defense2': 6.947122903633701}

In [6]:
# Test statistic for randomizing the scores.  Are the probabilities and RMS consistent?

import random

o1 = 9.0
d1 = 7.0
o2 = 8.0
d2 = 7.0

o1 = 10.
d1 = 8.
o2 = 9.
d2 = 7.

testSets = []
testSets.append({'o1': 9., 'd1': 7., 'o2': 8., 'd2':7.})
testSets.append({'o2': 9., 'd2': 7., 'o1': 8., 'd1':7.})
testSets.append({'o1': 10., 'd1': 8., 'o2': 9., 'd2':7.})
testSets.append({'o1': 10., 'd1': 8., 'o2': 9.8, 'd2':7.1})
for testSet in testSets:
    print("Test set "+str(testSet))
    o1 = testSet["o1"]
    d1 = testSet["d1"]
    o2 = testSet["o2"]
    d2 = testSet["d2"]
    
    totalScore1 = 0
    totalScore2 = 0
    spreadSquareError = 0
    gameCount = 0
    winCount = 0
    averageScore = gameSimulator.GetScoreAndProbability(o1, d1, o2, d2, 0.5)
    for index in range(10000):
        score = {}
        score = gameSimulator.GetRandomizedScore(o1, d1, o2, d2, 0.5)
        if (score["score1"] != score["score2"]): # skip ties
            totalScore1 += score["score1"]
            totalScore2 += score["score2"]
            spreadSquareError += (averageScore["score1"]-averageScore["score2"]-score["score1"]+score["score2"])**2
            gameCount += 1
            if (score["score1"] > score["score2"]):
                winCount += 1 

    spreadRms = math.sqrt(spreadSquareError/gameCount)
    averageScore1 = totalScore1/gameCount
    averageScore2 = totalScore2/gameCount
    averageProbability = winCount/gameCount
    print("   Average score"+str(averageScore))
    print("   Average from randomized scores score1="+str(averageScore1)+" score2="+str(averageScore2)
          +" prob="+str(averageProbability)+" spreadRms="+str(spreadRms))

Test set {'o1': 9.0, 'd1': 7.0, 'o2': 8.0, 'd2': 7.0}
   Average score{'score1': 59.30000000000001, 'score2': 40.900000000000006, 'probability': 0.8884145530642259, 'scoreRms': 15.104999999999997}
   Average from randomized scores score1=59.29802731411229 score2=40.73849266565503 prob=0.8923621648963075 spreadRms=15.038247645952815
Test set {'o2': 9.0, 'd2': 7.0, 'o1': 8.0, 'd1': 7.0}
   Average score{'score1': 40.900000000000006, 'score2': 59.30000000000001, 'probability': 0.1115854469357741, 'scoreRms': 15.104999999999997}
   Average from randomized scores score1=40.579528196820895 score2=59.40396881644224 prob=0.10711754581350613 spreadRms=15.261730343604444
Test set {'o1': 10.0, 'd1': 8.0, 'o2': 9.0, 'd2': 7.0}
   Average score{'score1': 78.1, 'score2': 42.099999999999994, 'probability': 0.9874655234980421, 'scoreRms': 16.069000000000003}
   Average from randomized scores score1=77.99469097465692 score2=42.02854853250526 prob=0.9879795652609437 spreadRms=16.186581145987333
Test set

In [7]:
class GameSeason:
    """Class for tracking team matchups for one season"""
    
    def __init__(self, seasonYear, roundNames, gameSimulator=None):
        self.seasonYear = seasonYear
        self.roundNames = roundNames # A round is a week or other interval over which results and predictions are made.
        self.seasonInfo = {}
        self.gameIndexByTeamAndRound = {}
        self.roundIndex = {}
        self.roundCount = 0
        self.gameSimulator = gameSimulator
        for roundName in self.roundNames:
            self.seasonInfo[roundName] = [] # array containing Game objects
            self.roundIndex[roundName] = self.roundCount
            self.roundCount += 1
        self.currentRound = 0
        
    def AddGame(self, team1, team2, roundName, homeField, generateScore=None, score1=None, score2=None):
        if (generateScore == None):
            # Default generateScore to True if gameSimulator has been provided.
            generateScore = not (gameSimulator == None)
        if (generateScore):
            actualPower1 = team1.GetPowerActual(year)
            actualPower2 = team2.GetPowerActual(year)
            scoreAndProbability = gameSimulator.GetRandomizedScore(
                actualPower1["offense"], actualPower1["defense"], 
                actualPower2["offense"], actualPower2["defense"], homeField)
            #for key in scoreAndProbability.keys():
            #    print("::: key " + key + "  = " + scoreAndProbability[key])
            #print("::: score1 "+str(scoreAndProbability["score1"]))
            score1 = int(scoreAndProbability["score1"]+0.5)
            score2 = int(scoreAndProbability["score2"]+0.5)
        self.seasonInfo[roundName].append(Game(team1, team2, roundName, homeField, score1, score2))
        gameIndex = len(self.seasonInfo[roundName]) - 1
        if (self.gameIndexByTeamAndRound.get(roundName) == None):
            self.gameIndexByTeamAndRound[roundName] = {}
        # Note that a team can only have one game per round.
        self.gameIndexByTeamAndRound[roundName][team1] = gameIndex
        self.gameIndexByTeamAndRound[roundName][team2] = gameIndex
        
    def PrintSeason(self):
        print("Schedule for season "+str(self.seasonYear))
        countString = ""
        for iRound in range(self.roundCount):
            print("Round "+self.roundNames[iRound])
            conferenceCount = 0
            divisionCount = 0
            nondivisionCount = 0
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
                    game.PrintGame(self.roundNames[iRound].ljust(6," ")+" ")
                    if (game.team1Object.GetConferenceName(self.seasonYear) == game.team2Object.GetConferenceName(self.seasonYear)):
                        conferenceCount += 1
                    elif (game.team1Object.GetDivisionName(self.seasonYear) == game.team2Object.GetDivisionName(self.seasonYear)):
                        divisionCount += 1
                    else:
                        nondivisionCount += 1
                countString += self.roundNames[iRound].ljust(6,' ')+" counts: conference="+str(conferenceCount).ljust(3,' ')+" division="+str(divisionCount).ljust(3,' ')+" nondivision="+str(nondivisionCount).ljust(3,' ')+"\n"
        print(countString, end='')

    def GetCurrentPower(self, teamObject, year, roundName):
        fitPower = teamObject.GetFitPower(year,roundName)
        while (fitPower == None):
            roundIndex = self.roundIndex[roundName]
#            print("::: GCP roundName "+roundName+" roundIndex "+str(roundIndex))
            if roundIndex > 0:
                roundName = self.roundNames[roundIndex-1]
#                print("::: GCP newRoundName "+roundName)
                fitPower = teamObject.GetFitPower(year,roundName)
            else:
                fitPower = {}
                fitPower["offense"] = teamObject.GetOffensePower(year-1)
                fitPower["defense"] = teamObject.GetDefensePower(year-1) 
        return fitPower

    def FitPowerForCurrentRound(self, learningRate=0):
        print("Fitting power for season "+str(self.seasonYear)+ ", round "+self.roundNames[self.currentRound])
        
        # Note that it is assumed that the current power for each team is synced to currentRound
        
        # Structure to store fit power for each game.
        # powerPerTeamToDate[teamId][gameId][{offenseAdjusted, defenseAdjusted, gameType (conference, nonconfence, nondivision)}]
        powerPerTeamToDate = {}

        # For each round from beginning to this round, for each game, compute fit power (starting from current power).
        for iRound in range(self.currentRound+1):
#            print("::: fitPower for round "+self.roundNames[iRound])
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
#                    game.PrintGame() # :::
                    # Perform fit of power to match actual score and store results in PowerPerTeamToDate
                    gameType = "TBD"
                    if (game.team1Object.GetConferenceName(year) == game.team2Object.GetConferenceName(year)):
                        gameType = "conference"
                    elif (game.team1Object.GetDivisionName(year) == game.team2Object.GetDivisionName(year)):
                        gameType = "nonconference"
                    else:
                        gameType = "nondivision"
                    # For computing new fit to power, start with power from previous year for each call to FitPowerForCurrentRound.
                    team1Power = {}
                    team1Power["offense"] = game.team1Object.GetOffensePower(year-1)
                    team1Power["defense"] = game.team1Object.GetDefensePower(year-1)
                    team2Power = {}
                    team2Power["offense"] = game.team2Object.GetOffensePower(year-1)
                    team2Power["defense"] = game.team2Object.GetDefensePower(year-1)

                    # Note that predictions for the coming week should use the latest fit power (i.e. the best currently derived 
                    # power for the season to-date)
                    #team1Power = self.GetCurrentPower(game.team1Object, year, self.roundNames[currentRound])
                    #team2Power = self.GetCurrentPower(game.team2Object, year, self.roundNames[currentRound])

#                    expectedResult = gameSimulator.GetScoreAndProbability(team1Power["offense"], team1Power["defense"], 
#                                                                     team2Power["offense"], team2Power["defense"], 
#                                                                     game.homeField) # :::
#                    print("::: expected "+str(expectedResult))
                    adjustedPower = gameSimulator.AdjustPowerToFitActualScore(team1Power["offense"], team1Power["defense"], 
                                                                     team2Power["offense"], team2Power["defense"], 
                                                                     game.homeField, game.score1, game.score2)
                    if (powerPerTeamToDate.get(game.team1Object.teamName) == None):
                        powerPerTeamToDate[game.team1Object.teamName] = {}
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId] = {}
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId]["offenseAdjusted"] = adjustedPower["offense1"]
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId]["defenseAdjusted"] = adjustedPower["defense1"]
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId]["offenseBase"] = team1Power["offense"]
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId]["defenseBase"] = team1Power["defense"]
                    powerPerTeamToDate[game.team1Object.teamName][game.gameId]["gameType"] = gameType
                    if (powerPerTeamToDate.get(game.team2Object.teamName) == None):
                        powerPerTeamToDate[game.team2Object.teamName] = {}
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId] = {}
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId]["offenseAdjusted"] = adjustedPower["offense2"]
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId]["defenseAdjusted"] = adjustedPower["defense2"]
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId]["offenseBase"] = team2Power["offense"]
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId]["defenseBase"] = team2Power["defense"]
                    powerPerTeamToDate[game.team2Object.teamName][game.gameId]["gameType"] = gameType
#                    print ("::: t1 "+str(powerPerTeamToDate[game.team1Object.teamName][game.gameId])+" t2 "
#                           +str(powerPerTeamToDate[game.team2Object.teamName][game.gameId]))
                  
        meanSquarePowerError = {}
        meanSquarePowerError["offense"] = 0
        meanSquarePowerError["defense"]= 0
        meanSquarePowerError["count"] = 0                
        
        # For each team in PowerPerTeamToDate
        for team in powerPerTeamToDate.keys():
            # Average the power over all games for the team
            offenseTotal = 0
            defenseTotal = 0
            weightTotal = 0
            baseCount = 0
            numberOfGames = len(powerPerTeamToDate[team])
            for gameId in powerPerTeamToDate[team].keys():
                if (baseCount == 0):
                    baseCount = 1
                    weightTotal += 1 # Base power (power from last year) is always weighted as 1.
                    offenseTotal += powerPerTeamToDate[team][gameId]["offenseBase"]
                    defenseTotal += powerPerTeamToDate[team][gameId]["defenseBase"]
                gameWeight = gameSimulator.weights[powerPerTeamToDate[team][gameId]["gameType"]][numberOfGames-1]
                offenseTotal += powerPerTeamToDate[team][gameId]["offenseAdjusted"]*gameWeight
                defenseTotal += powerPerTeamToDate[team][gameId]["defenseAdjusted"]*gameWeight
                weightTotal += gameWeight
#                print("::: gameWeight "+str(gameWeight)+" gameType "+powerPerTeamToDate[team][gameId]["gameType"])
            offenseUpdated = offenseTotal/weightTotal
            defenseUpdated = defenseTotal/weightTotal
#            print("::: team "+team+" o&d "+str(offenseUpdated)+" "+str(defenseUpdated))
            
            # Update team power
            Team.teamObjectByName[team].SetFitPower(year, self.roundNames[self.currentRound], offenseUpdated, defenseUpdated)
            # If actual power given, update meanSquarePowerError
            actualPower = Team.teamObjectByName[team].GetPowerActual(year)
            meanSquarePowerError["offense"] += (offenseUpdated-actualPower["offense"])**2
            meanSquarePowerError["defense"] += (defenseUpdated-actualPower["defense"])**2
            meanSquarePowerError["count"] += 1
            
            if (learningRate > 0):
                # Update the deltas for each weight.
                #
                #   Cost function is the squared error of (actual-derived power)^2
                #
                #   cost for a given week = sum over each team_i of [actualOffense_i - (sum over each game_ij of weight_ij*adjustedOffense_ij)/totalWeight_i]^2
                #                                                   [actualDefense_i - (sum over each game_ij of weight_ij*adjustedDefense_ij)/totalWeight_i]^2
                #
                #   Update the weights as follows (e.g. like http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks Backpropagation Algorithm)
                #
                #      weight_lk_new = weight_lk - learningRate * d cost/d weight_lk
                #
                #      d cost/d weight_lk  = sum over each team_i of 
                #         2*[actualOffense_i - (sum over each game_ij of weight_ij*adjustedOffense_ij)/totalWeight_i]*[sum over each game_ij of -adjustedOffense_ij*d weight_ij/d weight_lk]
                #         + 2*[actualDefense_i - (sum over each game_ij of weight_ij*adjustedDefense_ij)/totalWeight_i]*[sum over each game_ij of -adjustedDefense_ij*d weight_ij/d weight_lk]
                #
                #         where d weight_ij/d weight_lk is 0 when ij != lk and d weight_lk/d weigh_lk = 1
                #
                #         note that assuming that totalWeight is approximately constant, i.e. each weight's change is small with respect to the total weight
                #
                #      d cost/d weight_lk  = sum over each team_i of 
                #         2*[actualOffense_i - (sum over each game_ij of weight_ij*adjustedOffense_ij)/totalWeight_i]*[sum over each game with weight_lk_of -adjustedOffense_ij/totalWeight_i]
                #         + 2*[actualDefense_i - (sum over each game_ij of weight_ij*adjustedDefense_ij)/totalWeight_i]*[sum over each game with weight_lk_of -adjustedDefense_ij/totalWeight_i]
                #
                #   To do the update, for each week w, add the following to the deltaWeight_lk:
                #
                #      For each team i:
                #         deltaWeight_lk -= [actualOffense_iw - (sum over each game_ij of weight_ij*adjustedOffense_ij)/totalWeight_iw]*[sum over each game_ij using weight_lk of adjustedOffense_ijw/totalWeight_i]
                #                         + [actualDefense_iw - (sum over each game_ij of weight_ij*adjustedDefense_ij)/totalWeight_iw]*[sum over each game_ij using weight_lk of adjustedDefense_ijw/totalWeight_i]
                for gameId in powerPerTeamToDate[team].keys():
                    gameSimulator.deltaWeights[powerPerTeamToDate[team][gameId]["gameType"]][numberOfGames-1] -= learningRate/weightTotal*(
                        (actualPower["offense"]-offenseUpdated)*powerPerTeamToDate[team][gameId]["offenseAdjusted"]
                        + (actualPower["defense"]-defenseUpdated)*powerPerTeamToDate[team][gameId]["defenseAdjusted"])
      
        # Compute and print RMS of power
        if (meanSquarePowerError["count"] > 0):
            # Print RMS
            offenseRms = math.sqrt(meanSquarePowerError["offense"]/meanSquarePowerError["count"])
            defenseRms = math.sqrt(meanSquarePowerError["defense"]/meanSquarePowerError["count"])
            print("::: RMS errors for round "+self.roundNames[self.currentRound]+" "+str(offenseRms)+" "+str(defenseRms))
      
    def RandomizeTeamPower(self, maxOffense, minOffense, maxDefense, minDefense):
        for iRound in range(len(self.roundNames)):
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
                    game.team1Object.SetPower(self.seasonYear, minOffense+(maxOffense-minOffense)*random.random(), minDefense+(maxDefense-minDefense)*random.random())

    def AdjustPowerAndLinearParametersFromScores(self, learningRate):
        # Before running this for the first time, it is recommended to RandomizeTeamPower and gameSimulator.RandomizeLinearParameters
        
        #   s1 = 0.5*(totCoefOff*(o1+o2) + totCoefDef*(d1+d2) + totCoef0 + spdCoefOff*(o1-o2) + spdCoefDef*(d1-d2) + 0.5*homeFieldAdvantage)
        #   s2 = 0.5*(totCoefOff*(o2+o1) + totCoefDef*(d2+d1) + totCoef0 + spdCoefOff*(o2-o1) + spdCoefDef*(d2-d1) - 0.5*homeFieldAdvantage)
        #
        # cost function = sum over all games_ij of (s1_ij - actualScore1_ij)^2 + (s2_ij - actualScore2_ij)^2
        #              = sum over all games_ij of 
        #                  (0.5*(totCoefOff*(off_i+off_j) + totCoefDef*(def_i+def_j) + totCoef0 + spdCoefOff*(off_i-off_j) + spdCoefDef*(def_i-def_j) + homeFieldAdvantage) - actualScore1_ij)^2
        #                + (0.5*(totCoefOff*(off_j+off_i) + totCoefDef*(def_j+def_i) + totCoef0 + spdCoefOff*(off_j-off_i) + spdCoefDef*(def_j-def_i) - homeFieldAdvantage) - actualScore2_ij)^2
        #
        #   d cost/d off_i = sum over games involving off_i of (s1_ij - actualScore1_ij)*(totCoefOff + spdCoefOff) + (s2_ij - actualScore2_ij)*(totCoefOff - spdCoefOff)
        #   d cost/d def_i = sum over games involving off_i of (s1_ij - actualScore1_ij)*(totCoefDef + spdCoefDef) + (s2_ij - actualScore2_ij)*(totCoefDef - spdCoefDef)
        #   d cost/d totCoefOff = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)*(off_i + off_j)
        #   d cost/d totCoefDef = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)*(def_i + def_j)
        #   d cost/d totCoef0 = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)
        #   d cost/d spdCoefOff = sum over games_ij of (s1_ij - actualScore1_ij)*(off_i - off_j) + (s2_ij - actualScore2_ij)*(off_j - off_i)
        #   d cost/d spdCoefDef = sum over games_ij of (s1_ij - actualScore1_ij)*(def_i - def_j) + (s2_ij - actualScore2_ij)*(def_j - def_i)
        #   d cost/d homeFieldAdvantage = sum over games_ij of (s1_ij - s2_ij - actualScore1_ij + actualScore2_ij)
        
        teamPowerDeltas = {}
        linearDeltas = {}
        # teamPowerDeltas[teamName][offense]
        # teamPowerDeltas[teamName][defense]
        # linearDeltas[totCoefOff]
        # linearDeltas[totCoefDef]
        # linearDeltas[totCoef0]
        # linearDeltas[spdCoefOff]
        # linearDeltas[spdCoefDef]
        # linearDeltas[homeFieldAdvantage]
        linearDeltas["totCoefOff"] = 0
        linearDeltas["totCoefDef"] = 0
        linearDeltas["totCoef0"] = 0
        linearDeltas["spdCoefOff"] = 0
        linearDeltas["spdCoefDef"] = 0
        linearDeltas["homeFieldAdvantage"] = 0
        linearDeltas["count"]= 0
        
        costOrig = 0
        for iRound in range(len(self.roundNames)):
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
                    if (teamPowerDeltas.get(game.team1Object.teamName) == None):
                        teamPowerDeltas[game.team1Object.teamName] = {}
                        teamPowerDeltas[game.team1Object.teamName]["offense"] = 0
                        teamPowerDeltas[game.team1Object.teamName]["defense"] = 0
                        teamPowerDeltas[game.team1Object.teamName]["count"] = 0
                    if (teamPowerDeltas.get(game.team2Object.teamName) == None):
                        teamPowerDeltas[game.team2Object.teamName] = {}
                        teamPowerDeltas[game.team2Object.teamName]["offense"] = 0
                        teamPowerDeltas[game.team2Object.teamName]["defense"] = 0
                        teamPowerDeltas[game.team2Object.teamName]["count"] = 0
                    power1 = game.team1Object.GetPower(self.seasonYear)
                    power2 = game.team2Object.GetPower(self.seasonYear)
                    scores = gameSimulator.GetScores(
                        power1["offense"], power1["defense"], power2["offense"], power2["defense"], game.homeField)
                    teamPowerDeltas[game.team1Object.teamName]["offense"] += ( 
                        (scores["score1"] - game.score1)*(gameSimulator.totCoefOff + gameSimulator.spdCoefOff)
                        + (scores["score2"] - game.score2)*(gameSimulator.totCoefOff - gameSimulator.spdCoefOff))
                    teamPowerDeltas[game.team1Object.teamName]["defense"] += ( 
                        (scores["score1"] - game.score1)*(gameSimulator.totCoefDef + gameSimulator.spdCoefDef)
                        + (scores["score2"] - game.score2)*(gameSimulator.totCoefDef - gameSimulator.spdCoefDef))
                    teamPowerDeltas[game.team2Object.teamName]["offense"] += ( 
                        (scores["score2"] - game.score2)*(gameSimulator.totCoefOff + gameSimulator.spdCoefOff)
                        + (scores["score1"] - game.score1)*(gameSimulator.totCoefOff - gameSimulator.spdCoefOff))
                    teamPowerDeltas[game.team2Object.teamName]["defense"] += ( 
                        (scores["score2"] - game.score2)*(gameSimulator.totCoefDef + gameSimulator.spdCoefDef)
                        + (scores["score1"] - game.score1)*(gameSimulator.totCoefDef - gameSimulator.spdCoefDef))
                    teamPowerDeltas[game.team1Object.teamName]["count"] += 1
                    teamPowerDeltas[game.team2Object.teamName]["count"] += 1
                    linearDeltas["totCoefOff"] += (scores["score1"] + scores["score2"] - game.score1 - game.score2) * (power1["offense"] + power2["offense"])
                    linearDeltas["totCoefDef"] += (scores["score1"] + scores["score2"] - game.score1 - game.score2) * (power1["defense"] + power2["defense"])
                    linearDeltas["totCoef0"] += scores["score1"] + scores["score2"] - game.score1 - game.score2
                    linearDeltas["spdCoefOff"] += ((scores["score1"] - game.score1) * (power1["offense"] - power2["offense"]) 
                                                 + (scores["score2"] - game.score2) * (power2["offense"] - power1["offense"]))
                    linearDeltas["spdCoefDef"] += ((scores["score1"] - game.score1) * (power1["defense"] - power2["defense"]) 
                                                 + (scores["score2"] - game.score2) * (power2["defense"] - power1["defense"]))
                    linearDeltas["homeFieldAdvantage"] += (scores["score1"] - scores["score2"] 
                                                           - game.score1 + game.score2)
                    linearDeltas["count"] += 1
                    print("::: hfa s1-sa1 "+str(scores["score1"])+"-"+str(game.score1)+ " sa2-s2 "
                          +str(game.score2)+"-"+str(scores["score2"])+" running ave "
                          +str(linearDeltas["homeFieldAdvantage"]/linearDeltas["count"]))
                    costOrig += (scores["score1"] - game.score1)**2 + (scores["score2"] - game.score2)**2
        #gameSimulator.totCoefOff -= learningRate * linearDeltas["totCoefOff"]/linearDeltas["count"]
        #gameSimulator.totCoefDef -= learningRate * linearDeltas["totCoefDef"]/linearDeltas["count"]
        #gameSimulator.totCoef0 -= learningRate * linearDeltas["totCoef0"]/linearDeltas["count"]
        #gameSimulator.spdCoefOff -= learningRate * linearDeltas["spdCoefOff"]/linearDeltas["count"]
        #gameSimulator.spdCoefDef -= learningRate * linearDeltas["spdCoefDef"]/linearDeltas["count"]
        gameSimulator.homeFieldAdvantage -= learningRate * linearDeltas["homeFieldAdvantage"]/linearDeltas["count"]
    
        for iRound in range(len(self.roundNames)):
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
                    power1 = game.team1Object.GetPower(self.seasonYear)
                    power2 = game.team2Object.GetPower(self.seasonYear)
                    #power1["offense"] -= learningRate * teamPowerDeltas[game.team1Object.teamName]["offense"]/teamPowerDeltas[game.team1Object.teamName]["count"]
                    #power1["defense"] -= learningRate * teamPowerDeltas[game.team1Object.teamName]["defense"]/teamPowerDeltas[game.team1Object.teamName]["count"]
                    #power2["offense"] -= learningRate * teamPowerDeltas[game.team2Object.teamName]["offense"]/teamPowerDeltas[game.team2Object.teamName]["count"]
                    #power2["defense"] -= learningRate * teamPowerDeltas[game.team2Object.teamName]["defense"]/teamPowerDeltas[game.team2Object.teamName]["count"]
                    #game.team1Object.SetPower(self.seasonYear, power1["offense"], power1["defense"])
                    #game.team2Object.SetPower(self.seasonYear, power2["offense"], power2["defense"])

        costUpdated = 0
        for iRound in range(len(self.roundNames)):
            if (self.seasonInfo.get(self.roundNames[iRound]) != None):
                for game in self.seasonInfo[self.roundNames[iRound]]:
                    power1 = game.team1Object.GetPower(self.seasonYear)
                    power2 = game.team2Object.GetPower(self.seasonYear)
                    scores = gameSimulator.GetScores(
                        power1["offense"], power1["defense"], power2["offense"], power2["defense"], game.homeField)
                    costUpdated += (scores["score1"] - game.score1)**2 + (scores["score2"] - game.score2)**2
        print("AdjustPowerAndLinearParametersFromScores: cost reduction="+str(costOrig-costUpdated)+" current cost="+str(costUpdated))
        print("::: current linear parameters: totCoefOff="+str(gameSimulator.totCoefOff)+" totCoefDef="+str(gameSimulator.totCoefDef)+" totCoef0="+str(gameSimulator.totCoef0)+" spdCoefOff="+str(gameSimulator.spdCoefOff)+" spdCoefDef="+str(gameSimulator.spdCoefDef)+" homeFieldAdvantage="+str(gameSimulator.homeFieldAdvantage))
              
class Game:
    """Class to hold info on a game"""

    nextGameId = 0
    gameObjectById = {}
    
    def __init__(self, team1Object, team2Object, roundName, homeField, score1=None, score2=None):
        self.team1Object = team1Object
        self.team2Object = team2Object
        self.roundName = roundName
        self.homeField = homeField
        self.score1 = score1
        self.score2 = score2
        self.gameId = Game.nextGameId
        Game.nextGameId += 1
        Game.gameObjectById[self.gameId] = self    
        
    def SetResults(self, score1, score2):
        self.score1 = score1
        self.score2 = score2   
    
    def PrintGame(self, headerString=""):
        teamString = ""
        resultString = ""
        if self.homeField == 1:
            teamString += '{0} at {1}'.format(self.team1Object.teamName, self.team2Object.teamName)
            if (self.score1 != None):
                resultString = ' {0}-{1}'.format(str(self.score1), str(self.score2))
        elif self.homeField == 0:
            teamString += '{0} at {1}'.format(self.team2Object.teamName, self.team1Object.teamName)
            if (self.score1 != None):
                resultString = ' {0}-{1}'.format(str(self.score2), str(self.score1))
        else:
            teamString += '{0} vs {1}'.format(self.team1Object.teamName, self.team2Object.teamName)
            if (self.score1 != None):
                resultString = ' {0}-{1}'.format(str(self.score1), str(self.score2))
        print(headerString+teamString+resultString)

In [8]:
# What does a typical season of NCAA Div 1 look like?
#
# Note: counts are teams, not games, i.e. for number of games, divide the below by 2.
#
# 2017 colI_2017wk00out totals: all  16   conf   0   nonConf  14   nonDiv   2  pre1
# 2017 colI_2017wk01out totals: all 216   conf   8   nonConf 116   nonDiv  92  pre2
# 2017 colI_2017wk02out totals: all 218   conf  24   nonConf 144   nonDiv  50  week1
# 2017 colI_2017wk03out totals: all 220   conf  34   nonConf 158   nonDiv  28  week2
# 2017 colI_2017wk04out totals: all 218   conf 142   nonConf  70   nonDiv   6  week3
# 2017 colI_2017wk05out totals: all 214   conf 166   nonConf  46   nonDiv   2  week4  
# 2017 colI_2017wk06out totals: all 222   conf 194   nonConf  26   nonDiv   2  week5
# 2017 colI_2017wk07out totals: all 228   conf 212   nonConf  16   nonDiv   0  week6
# 2017 colI_2017wk08out totals: all 220   conf 206   nonConf  14   nonDiv   0  week7
# 2017 colI_2017wk09out totals: all 226   conf 216   nonConf   8   nonDiv   2  week8
# 2017 colI_2017wk10out totals: all 242   conf 222   nonConf  20   nonDiv   0  week9
# 2017 colI_2017wk11out totals: all 236   conf 222   nonConf  12   nonDiv   2  week10
# 2017 colI_2017wk12out totals: all 240   conf 214   nonConf  16   nonDiv  10  week11
# 2017 colI_2017wk13out totals: all 144   conf 116   nonConf  28   nonDiv   0  week12
# 2017 colI_2017wk14out totals: all  50   conf  38   nonConf  12   nonDiv   0  week13
# 2017 colI_2017wk15out totals: all  10   conf   0   nonConf  10   nonDiv   0  post1
# 2017 colI_2017wk16out totals: all  82   conf   0   nonConf  82   nonDiv   0  post2
# 2017 colI_2017wk17out totals: all   4   conf   2   nonConf   2   nonDiv   0  post3
#
# defAve, offAve - average for that divsion
# defRms, offRms - variation of the def & off inside that division
# defChange, offChange - RMS change for off & def between years
#
# Division IA  defAve  5.43 offAve  7.88 defRms 0.40 offRms 0.48 defChange 0.46 offChange 0.57
# Division IAA defAve  4.88 offAve  6.97 defRms 0.41 offRms 0.48 defChange 0.47 offChange 0.54

In [9]:
# Syntax : random.gauss(mu, sigma)
#
# Parameters :
# mu : mean
# sigma : standard deviation
#
# Returns : a random gaussian distribution floating number
#
# If we're trying to establish equivalency between RMS and standard deviation, .... if the mean is zero, as is often 
# the case in electrical signals, there is no difference between the RMS calculation and the 
# standard-deviation calculation.Jul 28, 2020

import random

defAve = [5.43, 4.88]
offAve = [7.88, 6.97]
defDivRms = [0.4, 0.41]
offDivRms = [0.48, 0.48]

defSeasonChange = [0.46, 0.47]
offSeasonChange = [0.57, 0.54]

In [10]:
numberTeamsInConference = 11
numberConferencesInDivision = 10
#numberTeamsInConference = 5
#numberConferencesInDivision = 2

weekNames = ["pre1", "pre2", 
               "week1", "week2", "week3", "week4", "week5", "week6", "week7", 
               "week8", "week9", "week10", "week11", "week12", "week13", 
               "post1", "post2", "post3"]

# Priority order for filling in conference games from sorting number of conference games in a given week.
conferenceWeeksList = ["week9", "week10", "week8", "week11", "week6", "week7", "week5", 
                       "week4", "week3", "week12", "week13", "week2", "week1", "pre2"]

nonConferenceWeekList = {}
nonDivisionWeekList = {}

nonConferenceWeekList["pre1"] = 14/2
nonConferenceWeekList["pre2"] = 116/2
nonConferenceWeekList["week1"] = 144/2
nonConferenceWeekList["week2"] = 158/2
nonConferenceWeekList["week3"] = 70/2
nonConferenceWeekList["week4"] = 46/2
nonConferenceWeekList["week5"] = 26/2
nonConferenceWeekList["week6"] = 16/2
nonConferenceWeekList["week7"] = 14/2
nonConferenceWeekList["week8"] = 8/2
nonConferenceWeekList["week9"] = 20/2
nonConferenceWeekList["week10"] = 12/2
nonConferenceWeekList["week11"] = 16/2
nonConferenceWeekList["week12"] = 28/2
nonConferenceWeekList["week13"] = 12/2

nonDivisionWeekList["pre1"] = 2/2
nonDivisionWeekList["pre2"] = 92/2
nonDivisionWeekList["week1"] = 50/2
nonDivisionWeekList["week2"] = 28/2
nonDivisionWeekList["week3"] = 6/2
nonDivisionWeekList["week4"] = 2/2
nonDivisionWeekList["week5"] = 2/2
nonDivisionWeekList["week6"] = 0
nonDivisionWeekList["week7"] = 0
nonDivisionWeekList["week8"] = 2/2
nonDivisionWeekList["week9"] = 0
nonDivisionWeekList["week10"] = 2/2
nonDivisionWeekList["week11"] = 10/2
nonDivisionWeekList["week12"] = 0
nonDivisionWeekList["week13"] = 0

divisionNames = ["D0", "D1"]

teamNamesByDivision = {}
teamNamesByConference = {}

# Set up the teams for an initial season

year = 2022

for iDiv in range(len(divisionNames)):
    teamNamesByDivision[divisionNames[iDiv]] = {}
    for iConf in range(numberConferencesInDivision):
        conferenceName = "C"+str(iConf)+"."+divisionNames[iDiv]
        teamNamesByConference[conferenceName] = {}
        for iTeam in range(numberTeamsInConference):
            teamName = "T"+str(iTeam).rjust(2,'0')+"."+conferenceName
            teamNamesByDivision[divisionNames[iDiv]][teamName] = 1
            teamNamesByConference[conferenceName][teamName] = 1
            teamObject = Team(teamName, year, conferenceName, divisionNames[iDiv])
            defensePower = random.gauss(defAve[iDiv], defDivRms[iDiv])
            offensePower = random.gauss(offAve[iDiv], offDivRms[iDiv])
            defensePrevious = random.gauss(defensePower, defSeasonChange[iDiv])
            offensePrevious = random.gauss(offensePower, offSeasonChange[iDiv])
            #print ("Team "+teamName+" def "+str(defensePower)+" off "+str(offensePower))
            teamObject.SetPowerActual(year, defensePower, offensePower)
            teamObject.SetPower(year, defensePrevious, offensePrevious) # start of the season, the current power is last year's power
            teamObject.SetPower(year-1, defensePrevious, offensePrevious)
            

In [11]:
# Set up the games for the season

gameSeason = GameSeason(year, weekNames, gameSimulator)

# List of games for pre-season and regular season.  Enforces only one matchup between any two given teams for the regular season.
roundNameForRegularSeasonMatchup = {} # Returns the round name for matchup of team1+team2 (games are added as team1+team2 as well as team2+team1).  Excludes tournaments (i.e. excludes repeat matchups in same season).

# Conference games
for conference in teamNamesByConference.keys():
    teamList1 = []
    for teamKey in teamNamesByConference[conference].keys():
        teamList1.append(str(teamKey))
    
    random.shuffle(teamList1)
    for team1 in teamList1:
        team1Object = Team.teamObjectByName.get(team1)
        homeCount = 0
        awayCount = 0
        teamList2 = []
        for teamKey in teamNamesByConference[conference].keys():
            teamList2.append(str(teamKey))
        random.shuffle(teamList2)
        for team2 in teamList2:
            if (team1 != team2):
                team2Object = Team.teamObjectByName.get(team2)
                roundName = roundNameForRegularSeasonMatchup.get(team1+"+"+team2)
                if (roundName == None):
                    iRound = 0
                    foundWeek = False
                    while (iRound < len(conferenceWeeksList) and not foundWeek):
                        # Find first available week for matchup
                        if (team1Object.GetOpponent(year, conferenceWeeksList[iRound]) == None \
                                and team2Object.GetOpponent(year, conferenceWeeksList[iRound]) == None):
                            if (awayCount >= homeCount):
                                gameSeason.AddGame(team1Object, team2Object, conferenceWeeksList[iRound], 1)
                                team1Object.SetOpponent(year, conferenceWeeksList[iRound], team2, 1)
                                team2Object.SetOpponent(year, conferenceWeeksList[iRound], team1, 0)
                                homeCount += 1
                            else:
                                gameSeason.AddGame(team2Object, team1Object, conferenceWeeksList[iRound], 1)
                                team1Object.SetOpponent(year, conferenceWeeksList[iRound], team2, 0)
                                team2Object.SetOpponent(year, conferenceWeeksList[iRound], team1, 1)
                                awayCount += 1
                            foundWeek = True
                            roundNameForRegularSeasonMatchup[team1+"+"+team2] = conferenceWeeksList[iRound]
                            roundNameForRegularSeasonMatchup[team2+"+"+team1] = conferenceWeeksList[iRound]
                        else:
                            iRound += 1  
                else:
                    homeField = team1Object.GetHomeField(year, roundName)
                    if (homeField == 1):
                        homeCount +=1
                    elif (homeField == 0):
                        awayCount += 1    
                                                
# Fill in non-division & non-conference games

for roundName in nonConferenceWeekList.keys():
    #year = gameSeason.seasonYear
    randomListOfTeamIds = Team.GetListOfTeamIds(year)
    random.shuffle(randomListOfTeamIds)
    numTeams = len(randomListOfTeamIds)
    nonConferenceCountRemaining = nonConferenceWeekList[roundName]
    nonDivisionCountRemaining = nonDivisionWeekList[roundName]
    idOffset1 = 0
    idOffset2 = 1
    while (nonConferenceCountRemaining > 0 or nonDivisionCountRemaining > 0):
        team1Object = Team.teamObjectById[randomListOfTeamIds[idOffset1]]
        team2Object = Team.teamObjectById[randomListOfTeamIds[(idOffset1+idOffset2) % numTeams]]
        matchupFound = False
        gotoNextTeam1 = False
        gotoNextTeam2 = False
        if (team1Object.GetOpponent(year, roundName) != None):
            # Team1 already has a game scheduled for this round.  Go to the next team1.
            gotoNextTeam1 = True
        elif (team2Object.GetOpponent(year, roundName) != None):
            # Team2 already has a game scheduled for this round.  Go to the next team2.
            gotoNextTeam2 = True
        else:
            # Team1 & team2 are both available.  See if they can fulfil the open matchups.
            sameDivision = team1Object.GetDivisionName(year) == team2Object.GetDivisionName(year)
            sameConference = team1Object.GetConferenceName(year) == team2Object.GetConferenceName(year)
            if (nonDivisionCountRemaining > 0 and not sameDivision):
                matchupFound = True
                nonDivisionCountRemaining -= 1
            elif (nonConferenceCountRemaining > 0 and sameDivision and not sameConference):
                matchupFound = True
                nonConferenceCountRemaining -= 1
            else:
                gotoNextTeam2 = True 
        if (matchupFound):
            if (random.random() < 0.5):
                gameSeason.AddGame(team1Object, team2Object, roundName, 1)
                team1Object.SetOpponent(year, roundName, team2, 1)
                team2Object.SetOpponent(year, roundName, team1, 0)
            else:
                gameSeason.AddGame(team2Object, team1Object, roundName, 1)
                team1Object.SetOpponent(year, roundName, team2, 0)
                team2Object.SetOpponent(year, roundName, team1, 1)
            gotoNextTeam1 = True
        if (gotoNextTeam2):
            idOffset2 += 1
            if (idOffset2 >= numTeams):
                # Have exhausted all possible team2s for this team1.  Go to next team1.
                gotoNextTeam1 = True
        if (gotoNextTeam1):
            idOffset1 += 1
            idOffset2 = 1
            if (idOffset1 >= numTeams):
                # Have exhausted all possible teams.  Quiting search for more games.
                nonConferenceCountRemaining = 0
                nonDivisionCountRemaining = 0

gameSeason.PrintSeason()

# Post season

Schedule for season 2022
Round pre1
pre1   T09.C9.D1 at T03.C1.D1 54-71
pre1   T07.C5.D1 at T03.C2.D1 42-36
pre1   T01.C4.D1 at T10.C1.D0 51-56
pre1   T03.C0.D1 at T02.C2.D1 47-52
pre1   T02.C1.D1 at T03.C7.D1 36-45
pre1   T02.C4.D0 at T06.C5.D0 69-62
pre1   T01.C0.D1 at T04.C8.D1 36-51
pre1   T05.C8.D0 at T06.C6.D0 47-48
Round pre2
pre2   T07.C1.D0 at T10.C1.D0 53-24
pre2   T04.C2.D0 at T01.C2.D0 75-76
pre2   T10.C9.D0 at T00.C9.D0 48-78
pre2   T06.C0.D1 at T02.C0.D1 44-53
pre2   T09.C1.D1 at T06.C1.D1 39-27
pre2   T00.C4.D1 at T10.C4.D1 47-58
pre2   T09.C7.D1 at T04.C7.D1 62-26
pre2   T04.C0.D1 at T01.C4.D1 62-27
pre2   T04.C8.D0 at T04.C0.D0 83-66
pre2   T08.C9.D0 at T00.C0.D1 58-54
pre2   T01.C1.D0 at T06.C0.D0 69-39
pre2   T09.C0.D1 at T05.C4.D1 25-46
pre2   T02.C1.D0 at T05.C0.D0 73-32
pre2   T04.C4.D0 at T09.C8.D1 63-50
pre2   T02.C2.D1 at T03.C5.D0 47-60
pre2   T00.C7.D1 at T05.C1.D0 45-49
pre2   T08.C4.D0 at T07.C7.D1 74-26
pre2   T03.C4.D1 at T03.C7.D0 56-71
pre2   T10.C7.D1 

In [12]:
# Football expected and actual score generators

# from bfm/code/bfm2003.f:
#  fblinear spd_coef_off, spd_coef_def, tot_coef_0, tot_coef_off, tot_coef_def, rms_coef_0, rms_coef_off, rms_coef_def
#                       - Linear coefficients to use (if fitflag=1).  If rms coefs are nonzero
#                         then linear rms is enabled (fitflagrms = 1).
    
# from bfm/col/colI21in:
# fblinear 18.4 17.6 -17 14.8 -9.6 16.173 1.324 -1.684

# from bfm/code/fbscore.f:
# c get the scores and rms's
#
#        s1 = FBscore(d1,o1,d2,o2,fitflag)
#        s2 = FBscore(d2,o2,d1,o1,fitflag)
#        rms = FBsrms(d1,o1,d2,o2,fitflagrms)
#
# compute the probability from the spread and rms
#        FBprob = FBgaussian((s1-s2)/rms)
#
#           s1 = 0.5*(tot_coef_off*(o1+o2) 
#     :           + tot_coef_def*(d1+d2) + tot_coef_0
#     :           + spd_coef_off*(o1-o2) + spd_coef_def*(d1-d2))
#
#           FBscore = max(0.00, s1)
#
#          FBsrms = max(1.0, rms_coef_0 + (o1+o2)*rms_coef_off
#     :                                 + (d1+d2)*rms_coef_def)
#
#        real gauss(0:47)
#        data gauss / 0.5000, 0.5199, 0.5398, 0.5596, 0.5793,
#     :               0.5987, 0.6179, 0.6368, 0.6654, 0.6736, 
#     :               0.6915, 0.7088, 0.7257, 0.7422, 0.7580,
#     :               0.7734, 0.7881, 0.8023, 0.8159, 0.8289,
#     :               0.8414, 0.8531, 0.8643, 0.8749, 0.8849,
#     :               0.8944, 0.9032, 0.9115, 0.9192, 0.9265,
#     :               0.9332, 0.9394, 0.9452, 0.9505, 0.9554,
#     :               0.9599, 0.9641, 0.9678, 0.9713, 0.9744,
#     :               0.9773, 0.9798, 0.9821, 0.9842, 0.9861,
#     :               0.9878, 0.9893, 0.9906 /
#        
#        ix = min(47,int(abs(x/0.05)))
#        ix2 = min(47,ix + 1)
#        frac = abs(x/0.05) - ix
#
#        ptem = (1.-frac)*gauss(ix) + frac*gauss(ix2)
#
#        if (x .lt. 0) ptem = 1. - ptem
#
#        FBgaussian = min( 0.99, max( 0.01, ptem) )


# from java_class/NetBeans-NN/NeuralNet/src/neuralnet/BfmData.java:
#
#	public static final double BFM_DATA_SPREAD_RMS = 10.0;
#	public static final double BFM_DATA_FUDGE_RMS = 2.699; // FIXME - Do the math to work this out.  Expect spread of 10 to give 70% prob of victory.
#
#	public static double GetSpreadFromProb(double prob)
#	{
#		return BFM_DATA_FUDGE_RMS*BFM_DATA_SPREAD_RMS*org.apache.commons.math3.special.Erf.erfcInv(2*(1.0-prob)); // 70% should be 10 points
#	}
#	
#	public static double GetProbFromSpread(double spread)
#	{
#		return -0.5*org.apache.commons.math3.special.Erf.erfc(spread/BFM_DATA_SPREAD_RMS/BFM_DATA_FUDGE_RMS)+1.0;
#	}

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.html?highlight=erf#scipy.special.erf
# scipy.special.erf
# scipy.special.erf(z) = <ufunc 'erf'>
# Returns the error function of complex argument.
# 
# It is defined as 2/sqrt(pi)*integral(exp(-t**2), t=0..z).
# 
# Parameters
# xndarray
# Input array.
# 
# Returns
# resndarray
# The values of the error function at the given points x.
#
# scipy.special.erfinv
# scipy.special.erfinv(x, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj]) = <ufunc 'erfinv'>
# Inverse of the error function.
# 
# Computes the inverse of the error function.
# 
# In the complex domain, there is no unique complex number w satisfying erf(w)=z. This indicates a true inverse function would have multi-value. When the domain restricts to the real, -1 < x < 1, there is a unique real number satisfying erf(erfinv(x)) = x.
# 
# Parameters
# yndarray
# Argument at which to evaluate. Domain: [-1, 1]
# 
# Returns
# erfinvndarray
# The inverse of erf of y, element-wise)
# 
# See also
# 
# erf
# Error function of a complex argument
# 
# erfc
# Complementary error function, 1 - erf(x)
# 
# erfcinv
# Inverse of the complementary error function
# 
# Examples
# 
# evaluating a float number
# 
# from scipy import special
# special.erfinv(0.5)
# 0.4769362762044698

# **Notes**
# The cumulative of the unit normal distribution is given by Phi(z) = 1/2[1 + erf(z/sqrt(2))].

# https://www.w3schools.com/python/ref_math_erf.asp
# The math.erf() method returns the error function of a number.
# This method accepts a value between - inf and + inf, and returns a value between - 1 to + 1.
# import math
## Print error function for different numbers
#print (math.erf(0.67))
#print (math.erf(1.34))
#print (math.erf(-6))


In [21]:
# Test new methods above
#teamObject = Team.teamObjectByName["T02.C0.D0"]
#year = 2022
#print("::: power-prev-season "+str(teamObject.GetOffensePower(year-1))+" "+str(teamObject.GetDefensePower(year-1)))
#fitPower = teamObject.GetFitPower(year,"week2")
#print("::: fitPower "+str(fitPower))
#print("::: power-cur-season "+str(teamObject.GetOffensePower(year))+" "+str(teamObject.GetDefensePower(year)))


#power = gameSeason.GetCurrentPower(teamObject, year, "week2")
#print("::: power-wk2 "+str(power))
#teamObject.SetFitPower(year,"pre2",8.,6.)
#power = gameSeason.GetCurrentPower(teamObject, year, "week2")
#print(":::-1 power-wk2 "+str(power))

if (False):
    gameSimulator.SetWeights()

    numIter = 5
    learningRate = 0.01
    print("Initial weights: "+str(gameSimulator.weights))
    for iter in range(numIter):
        print("Iteration "+str(iter))
        gameSeason.currentRound = 0
        gameSimulator.ResetDeltaWeights()
        for count in range(15):
            gameSeason.FitPowerForCurrentRound(learningRate)
            gameSeason.currentRound += 1
        print("Deltas: "+str(gameSimulator.deltaWeights))
        gameSimulator.ApplyDeltaWeights()
        print("Updated weights: "+str(gameSimulator.weights))
        print("")

#gameSeason.RandomizeTeamPower(5, 15, 0, 10)
#gameSimulator.RandomizeLinearParameters

gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)

gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)
gameSeason.AdjustPowerAndLinearParametersFromScores(0.01)


## Verify that result is the same if the fit is repeated:
#gameSeason.currentRound -= 1
#gameSeason.FitPowerForCurrentRound()
# yes:
#Fitting power for season 2022, round week13
#::: RMS errors for round week13 0.3129605777858281 0.2954001345292222
#Fitting power for season 2022, round week13
#::: RMS errors for round week13 0.3129605777858281 0.2954001345292222

# Next:
# cost function = sum over all games_ij of (s1_ij - actualScore1_ij)^2 + (s2_ij - actualScore2_ij)^2
#              = sum over all games_ij of 
#                  (0.5*(totCoefOff*(off_i+off_j) + totCoefDef*(def_i+def_j) + totCoef0 + spdCoefOff*(off_i-off_j) + spdCoefDef*(def_i-def_j)) - actualScore1_ij)^2
#                + (0.5*(totCoefOff*(off_j+off_i) + totCoefDef*(def_j+def_i) + totCoef0 + spdCoefOff*(off_j-off_i) + spdCoefDef*(def_j-def_i)) - actualScore2_ij)^2
#
#   d cost/d off_i = sum over games involving off_i of (s1_ij - actualScore1_ij)*(totCoefOff + spdCoefOff) + (s2_ij - actualScore2_ij)*(totCoefOff - spdCoefOff)
#   d cost/d def_i = sum over games involving off_i of (s1_ij - actualScore1_ij)*(totCoefDef + spdCoefDef) + (s2_ij - actualScore2_ij)*(totCoefDef - spdCoefDef)
#   d cost/d totCoefOff = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)*(off_i + def_j)
#   d cost/d totCoefDef = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)*(def_i + def_j)
#   d cost/d totCoef0 = sum over all games_ij of (s1_ij + s2_ij - actualScore1_ij - actualScore2_ij)
#   d cost/d spdCoefOff = sum over games_ij of (s1_ij - actualScore1_ij)*(off_i - off_j) + (s2_ij - actualScore2_ij)*(off_j - off_i)
#   d cost/d spdCoefDef = sum over games_ij of (s1_ij - actualScore1_ij)*(def_i - def_j) + (s2_ij - actualScore2_ij)*(def_j - def_i)


::: hfa s1-sa1 51.58542863875367-54 sa2-s2 71-51.92508189732169 running ave 16.66034674143198
::: hfa s1-sa1 54.7879338137824-42 sa2-s2 36-58.94239043975996 running ave 3.25294505772721
::: hfa s1-sa1 18.644048822848237-51 sa2-s2 56-64.43170648652287 running ave -11.42725584940674
::: hfa s1-sa1 51.356852543341034-47 sa2-s2 52-47.448382496921056 running ave -6.3433243754500594
::: hfa s1-sa1 44.05138983823463-36 sa2-s2 45-52.43827042455182 running ave -4.952035617623485
::: hfa s1-sa1 50.23700262282459-69 sa2-s2 62-70.93903804148336 running ave -8.743702251129365
::: hfa s1-sa1 50.157054075031716-36 sa2-s2 51-52.69203556010306 running ave -5.713884998835362
::: hfa s1-sa1 49.84012719173687-47 sa2-s2 48-69.81064685159446 running ave -7.37096433146314
::: hfa s1-sa1 70.16737350866562-53 sa2-s2 24-25.724696051571026 running ave -4.836115243845615
::: hfa s1-sa1 58.807066099551456-75 sa2-s2 76-49.38476889845568 running ave -3.310273999351476
::: hfa s1-sa1 64.64536800991826-48 sa2-s2 78-64

::: hfa s1-sa1 54.64693095516301-68 sa2-s2 65-71.1713236397017 running ave -3.1150939013541437
::: hfa s1-sa1 37.128082950637264-28 sa2-s2 27-58.452771335271265 running ave -3.141229404052484
::: hfa s1-sa1 50.802500119942316-60 sa2-s2 48-24.23351825751351 running ave -3.1171666170056342
::: hfa s1-sa1 50.72849749502046-48 sa2-s2 44-30.238411467946214 running ave -3.090562475019095
::: hfa s1-sa1 73.38007707853352-62 sa2-s2 35-23.360766860529043 running ave -3.055183243727735
::: hfa s1-sa1 29.470485303786766-23 sa2-s2 32-48.63312932706931 running ave -3.0648009173130593
::: hfa s1-sa1 34.567862469592605-56 sa2-s2 44-52.11818355255391 running ave -3.100592160780151
::: hfa s1-sa1 55.64785238048555-38 sa2-s2 54-42.511459260845804 running ave -3.0570874572978033
::: hfa s1-sa1 73.83350477892861-42 sa2-s2 54-12.25844214299836 running ave -2.9538096269834795
::: hfa s1-sa1 65.55325112406605-42 sa2-s2 26-48.18617042284781 running ave -2.947994162207973
::: hfa s1-sa1 62.11802049562274-69 sa

::: hfa s1-sa1 48.76025643914431-40 sa2-s2 70-81.95344132221727 running ave -2.712306653497745
::: hfa s1-sa1 76.70661957096019-63 sa2-s2 38-48.72039858399597 running ave -2.7053317114653557
::: hfa s1-sa1 44.491986163505636-65 sa2-s2 56-67.53028795159312 running ave -2.7411910880871435
::: hfa s1-sa1 58.346194200162955-47 sa2-s2 52-33.256663704586614 running ave -2.7011047369471397
::: hfa s1-sa1 53.78752778470167-37 sa2-s2 51-62.28942036771083 running ave -2.69110569773502
::: hfa s1-sa1 39.39236445413951-33 sa2-s2 70-66.98626679450702 running ave -2.6763709798819537
::: hfa s1-sa1 58.45428113960204-78 sa2-s2 46-38.48110756470667 running ave -2.687746229815314
::: hfa s1-sa1 55.4033971692321-66 sa2-s2 61-48.58765257146148 running ave -2.6822741874974696
::: hfa s1-sa1 57.732486758734815-51 sa2-s2 47-43.35199530520729 running ave -2.6664213165738957
::: hfa s1-sa1 35.08728485235772-44 sa2-s2 54-35.99977799441438 running ave -2.652174130907814
::: hfa s1-sa1 59.102914429891825-65 sa2-s

::: hfa s1-sa1 66.47974223713469-55 sa2-s2 35-32.08656571375037 running ave -2.34182101336356
::: hfa s1-sa1 83.56838100800968-84 sa2-s2 64-47.72191462786867 running ave -2.325331178917953
::: hfa s1-sa1 53.483256259206556-69 sa2-s2 59-49.90484447059886 running ave -2.3290415566647593
::: hfa s1-sa1 65.42331825125316-62 sa2-s2 61-74.34527101182161 running ave -2.335912969518971
::: hfa s1-sa1 71.30127031262111-69 sa2-s2 61-55.26562862126569 running ave -2.3265354336592288
::: hfa s1-sa1 53.086100589661505-87 sa2-s2 51-61.68951387626819 running ave -2.364725928558007
::: hfa s1-sa1 70.27770656514252-44 sa2-s2 27-26.90633436806107 running ave -2.338790821946419
::: hfa s1-sa1 37.589727931686866-19 sa2-s2 62-77.51013144285254 running ave -2.3339049902865625
::: hfa s1-sa1 45.740746170150004-42 sa2-s2 60-48.22713412760892 running ave -2.3178261461128438
::: hfa s1-sa1 55.65838094182901-70 sa2-s2 84-53.074888516465535 running ave -2.3008132581097147
::: hfa s1-sa1 53.358552759948-70 sa2-s2 

::: hfa s1-sa1 66.45852677783702-72 sa2-s2 42-39.80407955170795 running ave -2.728341985508954
::: hfa s1-sa1 69.73290674286088-54 sa2-s2 47-25.548630216265952 running ave -2.6993146265910597
::: hfa s1-sa1 24.639978074235486-39 sa2-s2 62-85.75075292146306 running ave -2.7250496994258246
::: hfa s1-sa1 37.86888218669344-45 sa2-s2 48-46.04003795036482 running ave -2.7268261017963735
::: hfa s1-sa1 43.34492546491244-53 sa2-s2 63-29.335950150156705 running ave -2.7074242139759437
::: hfa s1-sa1 58.32900901424255-38 sa2-s2 34-9.411348419485938 running ave -2.6728882568992707
::: hfa s1-sa1 47.96730168638476-49 sa2-s2 21-39.788358324508195 running ave -2.6853144658711723
::: hfa s1-sa1 21.687165310765206-51 sa2-s2 79-70.02819697515675 running ave -2.6980991995413537
::: hfa s1-sa1 68.82735764053244-43 sa2-s2 20-59.47575155186691 running ave -2.7060227123574125
::: hfa s1-sa1 68.27507538802074-39 sa2-s2 62-45.95295703915841 running ave -2.671295206167087
::: hfa s1-sa1 49.87115242816618-71 s

::: hfa s1-sa1 60.603849075357545-58 sa2-s2 42-52.27481304341546 running ave -2.5683789355933713
::: hfa s1-sa1 37.58749088645168-56 sa2-s2 23-27.460044728772882 running ave -2.583772548107949
::: hfa s1-sa1 68.29478250869079-63 sa2-s2 56-40.18583044816921 running ave -2.5658235143135326
::: hfa s1-sa1 52.302032298706926-66 sa2-s2 89-74.45279828582638 running ave -2.563238308009828
::: hfa s1-sa1 55.808238737021284-60 sa2-s2 33-42.867069437050304 running ave -2.5719339149629437
::: hfa s1-sa1 71.88462054818078-66 sa2-s2 56-51.203295833059855 running ave -2.5619163347436817
::: hfa s1-sa1 47.79930621319004-78 sa2-s2 104-80.37444055149642 running ave -2.564947466166312
::: hfa s1-sa1 37.957108323881265-16 sa2-s2 68-77.71770539966681 running ave -2.553774371532062
::: hfa s1-sa1 43.69101364648864-59 sa2-s2 46-49.43350944024746 running ave -2.5659830603874365
::: hfa s1-sa1 38.39179130919487-60 sa2-s2 73-60.303168454463645 running ave -2.570764819305961
::: hfa s1-sa1 63.14047711279124-99 

::: hfa s1-sa1 82.45392746016338-44 sa2-s2 36-37.969407514016574 running ave -2.3122302563592174
::: hfa s1-sa1 41.0519831604613-35 sa2-s2 49-61.2597673089249 running ave -2.3458126174980483
::: hfa s1-sa1 58.16706922816709-78 sa2-s2 84-66.42684725273368 running ave -2.345077279096925
::: hfa s1-sa1 67.27281410878462-29 sa2-s2 69-48.974321876505265 running ave -1.8311487239157698
::: hfa s1-sa1 43.44747835434712-71 sa2-s2 61-73.63901824575026 running ave -2.153504952213983
::: hfa s1-sa1 32.25194020997207-40 sa2-s2 54-41.13860739957327 running ave -2.092947970858877
::: hfa s1-sa1 66.36091302966031-79 sa2-s2 28-41.32698688296968 running ave -2.290246531870864
::: hfa s1-sa1 51.466569064734436-100 sa2-s2 23-30.05163094541076 running ave -2.7270892806315645
::: hfa s1-sa1 45.566544481801465-28 sa2-s2 56-48.188674133477384 running ave -2.4985936738920875
::: hfa s1-sa1 54.441601737216416-45 sa2-s2 28-44.68067139406385 running ave -2.5368233189159213
::: hfa s1-sa1 62.95076755081885-70 sa2

In [16]:
# Test 

# Sample BFM

o1 = 9.0
d1 = 7.0
o2 = 8.0
d2 = 7.0

#  fblinear , , , , , , , 
#                       - Linear coefficients to use (if fitflag=1).  If rms coefs are nonzero
#                         then linear rms is enabled (fitflagrms = 1).
    
# from bfm/col/colI21in:
# fblinear 
spd_coef_off = 18.4
spd_coef_def = 17.6 
tot_coef_0 = -17 
tot_coef_off = 14.8 
tot_coef_def = -9.6 
rms_coef_0 = 16.173 
rms_coef_off = 1.324 
rms_coef_def = -1.684

s1 = 0.5*(tot_coef_off*(o1+o2) + tot_coef_def*(d1+d2) + tot_coef_0 + spd_coef_off*(o1-o2) + spd_coef_def*(d1-d2))
s2 = 0.5*(tot_coef_off*(o2+o1) + tot_coef_def*(d2+d1) + tot_coef_0 + spd_coef_off*(o2-o1) + spd_coef_def*(d2-d1))

spread = s1 - s2

rms = rms_coef_0 + (o1+o2)*rms_coef_off + (d1+d2)*rms_coef_def

print("s1="+str(s1)+" s2="+str(s2)+" rms="+str(rms)+" spread/rms="+str(spread/rms)+" x="+str(spread/rms/0.05))
# x=24 => 0.8849

import math
from scipy import special
prob = 0.5*(1+special.erf(spread/rms/math.sqrt(2)))
print("prob="+str(prob))

# s1=59.30000000000001 s2=40.900000000000006 rms=15.104999999999997 spread/rms=1.218139688844754 x=24.362793776895078
# prob=0.8884145530642259
# .885+ from BFM and .888 from erf <-- match

totalScore1 = 0
totalScore2 = 0
spreadSquareError = 0
gameCount = 0
winCount = 0
averageScore = gameSimulator.GetScoreAndProbability(o1, d1, o2, d2, 0.5)
for index in range(10000):
    score = {}
    score = gameSimulator.GetRandomizedScore(o1, d1, o2, d2, 0.5)
    if (score["score1"] != score["score2"]): # skip ties
        totalScore1 += score["score1"]
        totalScore2 += score["score2"]
        spreadSquareError += (averageScore["score1"]-averageScore["score2"]-score["score1"]+score["score2"])**2
        gameCount += 1
        if (score["score1"] > score["score2"]):
            winCount += 1 

spreadRms = math.sqrt(spreadSquareError/gameCount)
averageScore1 = totalScore1/gameCount
averageScore2 = totalScore2/gameCount
averageProbability = winCount/gameCount
print("Average score"+str(averageScore))
print("Average from randomized scores score1="+str(averageScore1)+" score2="+str(averageScore2)
      +" prob="+str(averageProbability)+" spreadRms="+str(spreadRms)
      +" spread/rms="+str((averageScore1-averageScore2)/spreadRms))


s1=59.30000000000001 s2=40.900000000000006 rms=15.104999999999997 spread/rms=1.218139688844754 x=24.362793776895078
prob=0.8884145530642259
Average score{'score1': 59.30000000000001, 'score2': 40.900000000000006, 'probability': 0.8884145530642259, 'scoreRms': 15.104999999999997}
Average from randomized scores score1=59.29787881863392 score2=40.69856896376738 prob=0.8948543590784532 spreadRms=15.04708763315103 spread/rms=1.2360737378766522


In [ ]:
# Create training and validation data.
import random
import numpy.random
from numpy.random import default_rng
dataSetSize = 2500
numBins = 20
binSize = 1./numBins
randomSeed = 20210905
random.seed(randomSeed)
default_rng(randomSeed)
allowContinuousOutputValues = True

In [ ]:
# Create the training and validation arrays, filling in with random values which will then be scaled/adjusted.
X_train = default_rng().random((dataSetSize,1))
X_valid = default_rng().random((dataSetSize,1))
result_train = default_rng().random((dataSetSize))
result_valid = default_rng().random((dataSetSize))
y_train = np.empty((dataSetSize,numBins))
y_valid = np.empty((dataSetSize,numBins))
y_probs = np.arange(start=0.0, stop=1.0, step=binSize)

# Scale X data so that the forecast is for the favored team (i.e. preditions range from 0.5 to 1)
X_train = 0.5*X_train + 0.5
X_valid = 0.5*X_valid + 0.5

# Finish setting the result, which is expected+pertubation.
result_train = result_train + X_train[:,0] - 0.5
result_valid = result_valid + X_valid[:,0] - 0.5

nonBinaryLabelsAllowed = allowContinuousOutputValues # Multioutput target data is not supported with label binarization

# Fill in y according to result > 0
for index in range(dataSetSize):
    for pIndex in range(numBins):
        if (result_train[index] >= y_probs[pIndex]):
            y_train[index,pIndex] = 1.0
        else:
            #y_train[index,pIndex] = -1.0
            y_train[index,pIndex] = 0.0
        if (result_valid[index] >= y_probs[pIndex]):
            y_valid[index,pIndex] = 1.0
        else:
            #y_valid[index,pIndex] = -1.0
            y_valid[index,pIndex] = 0.0
    # Set the value at the transition point
    if (nonBinaryLabelsAllowed):
        iResid = int(result_train[index]/binSize)
        if (iResid >= 0 and iResid < numBins):
            y_train[index,iResid] = result_train[index]/binSize - iResid
        iResid = int(result_valid[index]/binSize)
        if (iResid >= 0 and iResid < numBins):
            y_valid[index,iResid] = result_valid[index]/binSize - iResid
print(dataSetSize)        
print(result_train[0:10])
print(y_train[0:10,:])

In [ ]:
# ##################### Build the neural network model #######################
# This script supports three types of models. For each one, we define a
# function that takes a Theano variable representing the input and returns
# the output layer of a neural network model built in Lasagne.
dropout1 = 0.01 # was 0.2
dropout2 = 0.025 # was 0.5

def build_mlp(input_var=None, num_units=20):
    # This creates an MLP of two hidden layers of num_units each, followed by
    # a softmax output layer of 10 units. It applies 20% dropout to the input
    # data and 50% dropout to the hidden layers.

    # Input layer, specifying the expected input shape of the network
    # (unspecified batchsize, 1 channel, 28 rows and 28 columns) and
    # linking it to the given Theano variable `input_var`, if any:
    # https://lasagne.readthedocs.io/en/latest/modules/layers.html
    l_in = lasagne.layers.InputLayer(shape=(None, 1),
                                     input_var=input_var)
    # The four numbers in the shape tuple represent, in order: (batchsize, channels, rows, columns)

    # Apply 20% dropout to the input data:
    l_in_drop = lasagne.layers.DropoutLayer(l_in, p=dropout1)

    # Add a fully-connected layer of num_units, using the tanh, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    # https://lasagne.readthedocs.io/en/latest/modules/nonlinearities.html
    l_hid1 = lasagne.layers.DenseLayer(
            l_in_drop, num_units=num_units,
            nonlinearity=lasagne.nonlinearities.tanh,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=dropout2)

    # Another layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1_drop, num_units=num_units,
            nonlinearity=lasagne.nonlinearities.rectify)

    # 50% dropout again:
    l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=dropout2)

    # Finally, we'll add the fully-connected output layer, of num_units tanh units:
    l_out = lasagne.layers.DenseLayer(
            l_hid2_drop, num_units=num_units,
            nonlinearity=lasagne.nonlinearities.tanh)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out

def build_custom_mlp(input_var=None, depth=2, width=800, drop_input=.2,
                     drop_hidden=.5):
    # By default, this creates the same network as `build_mlp`, but it can be
    # customized with respect to the number and size of hidden layers. This
    # mostly showcases how creating a network in Python code can be a lot more
    # flexible than a configuration file. Note that to make the code easier,
    # all the layers are just called `network` -- there is no need to give them
    # different names if all we return is the last one we created anyway; we
    # just used different names above for clarity.

    # Input layer and dropout (with shortcut `dropout` for `DropoutLayer`):
    network = lasagne.layers.InputLayer(shape=(None, 1),
                                        input_var=input_var)
    if drop_input:
        network = lasagne.layers.dropout(network, p=drop_input)
    # Hidden layers and dropout:
    nonlin = lasagne.nonlinearities.rectify
    for _ in range(depth):
        network = lasagne.layers.DenseLayer(
                network, width, nonlinearity=nonlin)
        if drop_hidden:
            network = lasagne.layers.dropout(network, p=drop_hidden)
    # Output layer:
    softmax = lasagne.nonlinearities.softmax
    network = lasagne.layers.DenseLayer(network, 10, nonlinearity=softmax)
    return network

In [ ]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
# Create the network

model='mlp'
num_epochs=500

# Prepare Theano variables for inputs and targets
# https://theano.readthedocs.io/en/rel-0.6rc3/library/tensor/basic.html
input_var = T.dmatrix('inputs')
target_var = T.dmatrix('targets')

# Create neural network model (depending on first command line parameter)
print("Building model and compiling functions...")
if model == 'mlp':
    network = build_mlp(input_var)
elif model.startswith('custom_mlp:'):
    depth, width, drop_in, drop_hid = model.split(':', 1)[1].split(',')
    network = build_custom_mlp(input_var, int(depth), int(width),
                                   float(drop_in), float(drop_hid))
else:
    print("Unrecognized model type %r." % model)

# Create a loss expression for training, i.e., a scalar objective we want
# to minimize:
# https://lasagne.readthedocs.io/en/latest/modules/objectives.html
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.squared_error(prediction, target_var)
loss = loss.mean()
# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step.
# https://lasagne.readthedocs.io/en/latest/modules/updates.html
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adam(
            loss, params, learning_rate=0.01)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.squared_error(test_prediction, target_var)
test_loss = test_loss.mean()

# As a bonus, also create an expression for the classification accuracy:
#test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
#                      dtype='float64')
    
# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
#val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
val_fn = theano.function([input_var, target_var], [test_loss, test_prediction])

In [ ]:
# NOTE about deterministic parameter from https://colinraffel.com/talks/next2015lasagne.pdf
# ... when using dropout we should define different losses:
# One for training, one for evaluation. The training loss should apply dropout,
# while the evaluation loss shouldn't. This is controlled by setting the deterministic kwarg.
#loss_train = objective.get_loss(target=true_output, deterministic=False)
#loss_eval = objective.get_loss(target=true_output, deterministic=True)

In [ ]:
# Launch the training loop.
batch_size = 250
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    #val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_valid, y_valid, batch_size, shuffle=False):
        inputs, targets = batch
        #err, acc = val_fn(inputs, targets)
        err, preds = val_fn(inputs, targets)
        val_err += err
        #val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    #print("  validation accuracy:\t\t{:.2f} %".format(
    #        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
#test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_valid, y_valid, batch_size, shuffle=False):
        inputs, targets = batch
        #err, acc = val_fn(inputs, targets)
        err, preds = val_fn(inputs, targets)
        test_err += err
        #test_acc += acc
        test_batches += 1
        print("X_valid")
        print(inputs[0:1])
        print("y_valid")
        print(targets[0:1])
        print("pred")
        print(preds[0:1])

print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
#print("  test accuracy:\t\t{:.2f} %".format(
#        test_acc / test_batches * 100))

# Optionally, you could now dump the network weights to a file like this:
# np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
#
# And load them again later on like this:
# with np.load('model.npz') as f:
#     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
# lasagne.layers.set_all_param_values(network, param_values)

In [ ]:
# Method to get the resultant probability from a probability confidence array.
def get_probability(prob_array):
    binSize = 1./prob_array.size
    iProb = prob_array.size - 1
    while prob_array[iProb] < 0.5 and iProb > 0:
        iProb -= 1
    prob = binSize*(iProb+prob_array[iProb])
    return prob

# Method to compute skill.
def compute_skill(predicted,actual):
    skill = 0.0
    numBins = predicted[0,:].size
    numSets = predicted[:,0].size
    binSize = 1./numBins
    goodExpectedTotal = np.zeros((3,numBins))
    for index in range(numSets):
        predictedProb = get_probability(predicted[index,:])
        actualProb = get_probability(actual[index,:])
        iBin = int(predictedProb/binSize+0.5)
        if (iBin > numBins - 1):
            iBin = numBins - 1
        if ((predictedProb > 0.5 and actualProb > 0.5) or (predictedProb < 0.5 and actualProb < 0.5) or (predictedProb == 0.5 and actualProb == 0.5)):
            goodExpectedTotal[0,iBin] += 1
        goodExpectedTotal[1,iBin] += predictedProb
        goodExpectedTotal[2,iBin] += 1
    return goodExpectedTotal

# Print skill histogram.
def print_histogram(goodExpectedTotal):
    numBins = goodExpectedTotal[0,:].size
    dProb = 1./numBins
    headerString   = "                 "
    totalString    = "total games:     "
    rightString    = "number right:    "
    expectedString = "number expected: "
    rightExpString = "right/expected:  "
    totalGames = 0
    totalGood = 0
    totalExpected = 0
    for iBin in range(int(0.5*numBins-1.5),numBins):
        iBegPct = int(100*iBin*dProb)
        iEndPct = iBegPct + int(100*dProb-1)
        headerString += ' {0:02d}'.format(iBegPct)
        headerString += '-{0:02d}%'.format(iEndPct)
        totalString += ' {:6d}'.format(int(goodExpectedTotal[2,iBin]+0.5))
        rightString += ' {:6d}'.format(int(goodExpectedTotal[0,iBin]+0.5))
        expectedString += ' {:6.1f}'.format(goodExpectedTotal[1,iBin])
        if (goodExpectedTotal[1,iBin] > 0.0):
            rightExpString += ' {:6.2f}'.format(goodExpectedTotal[0,iBin]/goodExpectedTotal[1,iBin])
        else:
            rightExpString += ' {:6.2f}'.format(0.0)
        totalGames += goodExpectedTotal[2,iBin]
        totalGood += goodExpectedTotal[0,iBin]
        totalExpected += goodExpectedTotal[1,iBin]
    headerString += "    all"
    totalString += ' {:6d}'.format(int(totalGames+0.5))
    rightString += ' {:6d}'.format(int(totalGood+0.5))
    expectedString += ' {:6.1f}'.format(totalExpected)
    if (totalExpected == 0.0):
        totalExpected = 1.0;
    rightExpString += ' {:6.2f}'.format(totalGood/totalExpected)
    print(headerString)
    print(totalString)
    print(rightString)
    print(expectedString)
    print(rightExpString)

In [ ]:
err, predictions_valid = val_fn(X_valid, y_valid)
trainGoodExpectedTotal = compute_skill(predictions_valid,y_valid)
print("Validation data:")
print_histogram(trainGoodExpectedTotal)